In [2]:
import numpy as np
import pandas as pd
import sys
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive

Mounted at /content/gdrive


In [4]:
!cp /mydrive/Sequence\ Learning/Text_Generation_using_LSTM/wonderland.txt /content
#weights on drive
#!cp /mydrive/Sequence\ Learning/Text_Generation_using_LSTM/weights-improvment-20-1.9774.hdf5 /content
#!cp /mydrive/Sequence\ Learning/Text_Generation_using_LSTM/weights-improvement-bigger-40-1.2682.hdf5 /content


In [5]:
filename = 'wonderland.txt'
raw_text = open(filename, 'r',encoding = 'utf-8').read()
raw_text = raw_text.lower()

In [6]:
print(raw_text[50:100])

ing to get very tired of sitting by her sister on 


In [27]:
#Convertinv Character into interger for the model.
chars = sorted(list(set(raw_text)))
chars_to_int = dict((c,i) for i, c in enumerate(chars))

In [28]:
print(chars_to_int)

{'\n': 0, ' ': 1, '!': 2, '(': 3, ')': 4, '*': 5, ',': 6, '-': 7, '.': 8, ':': 9, ';': 10, '?': 11, '[': 12, ']': 13, '_': 14, 'a': 15, 'b': 16, 'c': 17, 'd': 18, 'e': 19, 'f': 20, 'g': 21, 'h': 22, 'i': 23, 'j': 24, 'k': 25, 'l': 26, 'm': 27, 'n': 28, 'o': 29, 'p': 30, 'q': 31, 'r': 32, 's': 33, 't': 34, 'u': 35, 'v': 36, 'w': 37, 'x': 38, 'y': 39, 'z': 40, 'ù': 41, '—': 42, '‘': 43, '’': 44, '“': 45, '”': 46}


In [29]:
#Using Dataframe to see the integer assigned to characters
df = pd.DataFrame(list(chars_to_int.items()), columns=['Character', 'Integer'])
df.head(10)

,Character,Integer
0,\n,0
1,,1
2,!,2
3,(,3
4,),4
5,*,5
6,",",6
7,-,7
8,.,8
9,:,9


In [30]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Character:",n_chars)
print("Total Vaacb:",n_vocab)

Total Character: 144017
Total Vaacb: 47


###Around 150K characters are present and 47 different characters in the whole data.

In [40]:
#Preparing the Training Data
seq_length = 100
data_X = []
data_y = []
for i in range(0,n_chars - seq_length,1):
  seq_in = raw_text[i : i+seq_length]
  seq_out = raw_text[i+seq_length]
  data_X.append([chars_to_int[char] for char in seq_in])
  data_y.append(chars_to_int[seq_out])
n_patterns = len(data_X)
print("Total Patterns:",n_patterns)

Total Patterns: 143917


In [41]:
#rescaling the integers to the range 0-to-1 to make the patterns easier to learn by the LSTM network
#using the sigmoid activation function by default.
X = np.reshape(data_X,(n_patterns, seq_length,1))
X = X/float(n_vocab)
y = to_categorical(data_y)

In [42]:
model = Sequential()
model.add(LSTM(256,input_shape = (X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1],activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam')

In [43]:
filepath = 'weights-improvment-{epoch:02d}-{loss:.4f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'loss',save_best_only=True, verbose = 1,mode = 'min')
callback_list = [checkpoint]

In [44]:
model.fit(X,y,epochs = 20, batch_size = 128, callbacks = callback_list)

Epoch 1/20
1121/1125 [============================>.] - ETA: 0s - loss: 3.0288
Epoch 1: loss improved from inf to 3.02845, saving model to weights-improvment-01-3.0284.hdf5
1125/1125 [==============================] - 17s 13ms/step - loss: 3.0284
Epoch 2/20
  11/1125 [..............................] - ETA: 13s - loss: 2.8831

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1123/1125 [============================>.] - ETA: 0s - loss: 2.8074
Epoch 2: loss improved from 3.02845 to 2.80736, saving model to weights-improvment-02-2.8074.hdf5
1125/1125 [==============================] - 14s 12ms/step - loss: 2.8074
Epoch 3/20
1124/1125 [============================>.] - ETA: 0s - loss: 2.6887
Epoch 3: loss improved from 2.80736 to 2.68873, saving model to weights-improvment-03-2.6887.hdf5
1125/1125 [==============================] - 14s 13ms/step - loss: 2.6887
Epoch 4/20
1124/1125 [============================>.] - ETA: 0s - loss: 2.6072
Epoch 4: loss improved from 2.68873 to 2.60716, saving model to weights-improvment-04-2.6072.hdf5
1125/1125 [==============================] - 14s 13ms/step - loss: 2.6072
Epoch 5/20
1123/1125 [============================>.] - ETA: 0s - loss: 2.5391
Epoch 5: loss improved from 2.60716 to 2.53907, saving model to weights-improvment-05-2.5391.hdf5
1125/1125 [==============================] - 15s 13ms/step - loss: 2.5391
Epoch 6

In [45]:
filename = 'weights-improvment-20-1.9774.hdf5'
model.load_weights(filename)
model.compile(loss='categorical_crossentropy',optimizer = 'adam')

In [46]:
int_to_char = dict((i,c) for i, c in enumerate(chars))

In [47]:
start = np.random.randint(0,len(data_X)-1)
pattern = data_X[start]
print('Seed:')
print('\"',''.join([int_to_char[value] for value in pattern]), '\"')


Seed:
" aying anything more till the pigeon had finished.

“as if it wasn’t trouble enough hatching the eggs "


In [48]:
for i in range(1000):
  X = np.reshape(pattern, (1,len(pattern),1))
  X = X/float(n_vocab)
  prediction = model.predict(X,verbose = 0)
  index = np.argmax(prediction)
  result = int_to_char[index]
  seq_in = [int_to_char[value] for value in pattern]
  sys.stdout.write(result)
  pattern.append(index)
  pattern = pattern[1:len(pattern)]
print("\nDone.")

ee of thens ” said alice an an cxplyuul 
“oh  o aaner the mirtle gorse      beautirui      wou              fot in wou coo a foea tf ther.”

“i wooher wo tey i woole if the mors ou toaee”!” she monk turtle senlied torh a aaterplll
tene an i sinele and the had foo a lotte, and the woid ti the whrle thrne  the kad here to the toed a deel of theng sas an in afdine the was oo a lirtle oo the that whre a latge carl was anl aroiers, 
“hh mo hin hor then ”our eajesn,” said the caterpillar.

“ie iou the more tu here th the soin ”oul toen ” she said to herself, “io wou don’t keke to moke the mort oo the reaten.”

“i whnu h tound ”ou soon!” said the caterpillar.

“he moeh tonh yhu io wou can ” said the mors turtle. 
“whel io horsten ” she mock turtle sepeied an an offended tone. 
“ie ious bn a moee tare,” she hatter went on an an offende tone. 
“ie iou the more tu seat to toe toin of the soise ”hu,” shi said to herself, “and io the would wou doul toe toin a crea fo soen a doeat outee, and io ii 

The Model has tried predicting but it's not as we expected.

#Larger LSTM


In [33]:
model1 = Sequential()
model1.add(LSTM(256,input_shape=(X.shape[1],X.shape[2]),return_sequences=True))
model1.add(Dropout(0.2))
model1.add(LSTM(256))
model1.add(Dropout(0.2))
model1.add(Dense(y.shape[1],activation = 'softmax'))
model1.compile(loss = 'categorical_crossentropy',optimizer = 'adam')

In [13]:
filepath = 'weights-improvement-bigger-{epoch:02d}-{loss:.4f}.hdf5'

In [14]:
filepath = 'weights-improvement-bigger-{epoch:02d}-{loss:.4f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'loss',save_best_only=True, verbose = 1,mode = 'min')
callback_list = [checkpoint]

In [15]:
model1.fit(X,y,epochs = 50,batch_size=64, callbacks = callback_list)

Epoch 1/50
2247/2249 [============================>.] - ETA: 0s - loss: 2.8083
Epoch 1: loss improved from inf to 2.80824, saving model to weights-improvement-bigger-01-2.8082.hdf5
2249/2249 [==============================] - 46s 18ms/step - loss: 2.8082
Epoch 2/50
   7/2249 [..............................] - ETA: 39s - loss: 2.5501

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


2248/2249 [============================>.] - ETA: 0s - loss: 2.3853
Epoch 2: loss improved from 2.80824 to 2.38519, saving model to weights-improvement-bigger-02-2.3852.hdf5
2249/2249 [==============================] - 40s 18ms/step - loss: 2.3852
Epoch 3/50
2247/2249 [============================>.] - ETA: 0s - loss: 2.1794
Epoch 3: loss improved from 2.38519 to 2.17956, saving model to weights-improvement-bigger-03-2.1796.hdf5
2249/2249 [==============================] - 41s 18ms/step - loss: 2.1796
Epoch 4/50
2248/2249 [============================>.] - ETA: 0s - loss: 2.0511
Epoch 4: loss improved from 2.17956 to 2.05096, saving model to weights-improvement-bigger-04-2.0510.hdf5
2249/2249 [==============================] - 42s 19ms/step - loss: 2.0510
Epoch 5/50
2249/2249 [==============================] - ETA: 0s - loss: 1.9567
Epoch 5: loss improved from 2.05096 to 1.95671, saving model to weights-improvement-bigger-05-1.9567.hdf5
2249/2249 [==============================] - 42s 

In [38]:
#Selecting weights will lowest loss.
filename = 'weights-improvement-bigger-40-1.2682.hdf5'
model1.load_weights(filename)
model1.compile(loss='categorical_crossentropy',optimizer = 'adam')

In [35]:
int_to_char = dict((i,c) for i, c in enumerate(chars))

In [36]:
start = np.random.randint(0,len(data_X)-1)
pattern = data_X[start]
print('Seed:')
print('\"',''.join([int_to_char[value] for value in pattern]), '\"')

Seed:
" e i’ve got to?” (alice had no
idea what latitude was, or longitude either, but thought they were nic "


In [37]:
for i in range(1000):
  X = np.reshape(pattern, (1,len(pattern),1))
  X = X/float(n_vocab)
  prediction = model1.predict(X,verbose = 0)
  index = np.argmax(prediction)
  result = int_to_char[index]
  seq_in = [int_to_char[value] for value in pattern]
  sys.stdout.write(result)
  pattern.append(index)
  pattern = pattern[1:len(pattern)]
print("\nDone.")

e all of the laster and the larch hare was seeling at the court, and was going to see it all the table for tome minute the thought it was a little siarp her hands, and she was a little siarp her hands and then she was all to tpeak as the sabbit, and she was soon for tome minutes, and the thought it was oot a little siarp her and then, and then was sitting on the door, and the white rabbit were all talking at the court, and was going to see it all the time, and then was she was seriing things at the white rabbit as well as she could, and the tornd of the lock turtle said in a low voice,
“yhat a canlya whine wou fan any muster to be a grown of the sea.”

“i don’t know what i say it out of the sorgs!” said the king, 
“then the queen was the eeged the shat then the wery sathen of the sea,” the mock turtle seplied rizinl, and the white rabbit were all talking at the court, and was going to see it all the table for tome minute the thought it was a little siarp her hands, and she was a little

## Comparatively better result this time.